In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv('/Users/bobbymarriott/Desktop/ADS505/Big_Black_Money_Dataset.csv')

print(df.head(5))

  Transaction ID       Country  Amount (USD)   Transaction Type  \
0   TX0000000001        Brazil  3.267530e+06  Offshore Transfer   
1   TX0000000002         China  4.965767e+06    Stocks Transfer   
2   TX0000000003            UK  9.416750e+04    Stocks Transfer   
3   TX0000000004           UAE  3.864201e+05    Cash Withdrawal   
4   TX0000000005  South Africa  6.433784e+05     Cryptocurrency   

   Date of Transaction Person Involved      Industry Destination Country  \
0  2013-01-01 00:00:00     Person_1101  Construction                 USA   
1  2013-01-01 01:00:00     Person_7484  Luxury Goods        South Africa   
2  2013-01-01 02:00:00     Person_3655  Construction         Switzerland   
3  2013-01-01 03:00:00     Person_3226     Oil & Gas              Russia   
4  2013-01-01 04:00:00     Person_7975   Real Estate                 USA   

   Reported by Authority Source of Money  Money Laundering Risk Score  \
0                   True         Illegal                           